In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import GridSearchCV,train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, r2_score

# Lectura de los datos de train

In [3]:
path = 'continuous_train.csv'
data = pd.read_csv(path, sep = ',', na_filter = False)
data = data.set_index('Unnamed: 0')

X = data.drop(columns = ['y']).copy()
y_train = data[['y']].copy()


standarscaler = StandardScaler()
X_trained_scaled = standarscaler.fit_transform(X)

X_trained_scaled = pd.DataFrame(X_trained_scaled, index=X.index, columns=X.columns.values)

In [4]:
X_trained_scaled.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 120 entries, 24 to 93
Data columns (total 32 columns):
x1     120 non-null float64
x2     120 non-null float64
x3     120 non-null float64
x4     120 non-null float64
x5     120 non-null float64
x6     120 non-null float64
x7     120 non-null float64
x8     120 non-null float64
x9     120 non-null float64
x10    120 non-null float64
x11    120 non-null float64
x12    120 non-null float64
x13    120 non-null float64
x14    120 non-null float64
x15    120 non-null float64
x16    120 non-null float64
x17    120 non-null float64
x18    120 non-null float64
x19    120 non-null float64
x20    120 non-null float64
x21    120 non-null float64
x22    120 non-null float64
x23    120 non-null float64
x24    120 non-null float64
x25    120 non-null float64
x26    120 non-null float64
x27    120 non-null float64
x28    120 non-null float64
x29    120 non-null float64
x30    120 non-null float64
x31    120 non-null float64
x32    120 non-null float64

# Lectura Datos de Test

In [5]:
path = 'continuous_test.csv'
test = pd.read_csv(path, sep = ',', na_filter = False)
test = test.set_index('Unnamed: 0')

X_test = test.drop(columns = ['y']).copy()
y_test = test[['y']].copy()

X_test_scaled = standarscaler.transform(X_test)

X_test_scaled = pd.DataFrame(X_test_scaled, index=X_test.index, columns=X_test.columns.values)

In [6]:
def calculated_metrics(variablesSelected,X_trained_scaled,y_train,X_test_scaled,y_test):
    X_trained_scaled_aux = X_trained_scaled[variablesSelected].copy()
    X_test_scaled_aux = X_test_scaled[variablesSelected].copy()
    ## Regresion para cada cantidad de variables
    regr = LinearRegression()

    # Train the model using the training sets
    regr.fit(X_trained_scaled_aux,  y_train['y'].values)

    # Make predictions using the testing set
    y_pred = regr.predict(X_test_scaled_aux)

    y_pred = pd.DataFrame(y_pred, index=y_test.index, columns=["y_pred"])

    total = y_pred.join(y_test)
    total['y_menor_-1'] = (total['y']<-1).astype(int)
    total['y_pred_menor_-1'] = (total['y_pred']<-1).astype(int)
    total['good_class'] = (total['y_menor_-1']==total['y_pred_menor_-1']).astype(int)
    accuray =total['good_class'].sum()/len(total)
    auxRes= pd.DataFrame({'Variables seleccionadas': [variablesSelected], 'Mean squared error': [mean_squared_error(y_test, y_pred)], 'R2': [r2_score(y_test, y_pred)],'Accuray': [accuray]})
    return auxRes

In [7]:
resultado_scores = pd.DataFrame()

## Feature ranking with recursive feature elimination

### Cross Validation

In [8]:
from sklearn.feature_selection import RFE
# from sklearn.svm import SVR
resultados =  pd.DataFrame({'VariableName': X_trained_scaled.columns})
resultado_scores_ranking_cv = pd.DataFrame()
for i in range(1,15):
    for j in range(1,100):
        X_train, X_test_aux, y_train_aux, y_test_aux = train_test_split(X, y_train, test_size=0.2)
        standarscaler = StandardScaler()
        X_trained_scaled_aux = standarscaler.fit_transform(X_train)
        X_trained_scaled_aux = pd.DataFrame(X_trained_scaled_aux, index=X_train.index, columns=X_train.columns.values)
        
        X_test_scaled_aux = standarscaler.transform(X_test_aux)

        X_test_scaled_aux = pd.DataFrame(X_test_scaled_aux, index=X_test_aux.index, columns=X_test_aux.columns.values)

        name_col = f'Cantidad de variables: {i} Iteracion: {j}'
        rfe_selector = RFE(estimator=LinearRegression(), n_features_to_select=i, step=1, verbose=5)
        selector = rfe_selector.fit(X_trained_scaled_aux, y_train_aux['y'].values)
        resultados[name_col] = selector.support_.astype(int)

        variablesSelected = X_trained_scaled.columns[selector.support_].values
        auxRes= calculated_metrics(variablesSelected,X_trained_scaled_aux,y_train_aux,X_test_scaled_aux,y_test_aux)
        resultado_scores_ranking_cv = pd.concat([resultado_scores_ranking_cv, auxRes])
        print(i)
resultado_scores_ranking_cv.reset_index(inplace = True,drop = True)

Fitting estimator with 32 features.
Fitting estimator with 31 features.
Fitting estimator with 30 features.
Fitting estimator with 29 features.
Fitting estimator with 28 features.
Fitting estimator with 27 features.
Fitting estimator with 26 features.
Fitting estimator with 25 features.
Fitting estimator with 24 features.
Fitting estimator with 23 features.
Fitting estimator with 22 features.
Fitting estimator with 21 features.
Fitting estimator with 20 features.
Fitting estimator with 19 features.
Fitting estimator with 18 features.
Fitting estimator with 17 features.
Fitting estimator with 16 features.
Fitting estimator with 15 features.
Fitting estimator with 14 features.
Fitting estimator with 13 features.
Fitting estimator with 12 features.
Fitting estimator with 11 features.
Fitting estimator with 10 features.
Fitting estimator with 9 features.
Fitting estimator with 8 features.
Fitting estimator with 7 features.
Fitting estimator with 6 features.
Fitting estimator with 5 feature

Fitting estimator with 25 features.
Fitting estimator with 24 features.
Fitting estimator with 23 features.
Fitting estimator with 22 features.
Fitting estimator with 21 features.
Fitting estimator with 20 features.
Fitting estimator with 19 features.
Fitting estimator with 18 features.
Fitting estimator with 17 features.
Fitting estimator with 16 features.
Fitting estimator with 15 features.
Fitting estimator with 14 features.
Fitting estimator with 13 features.
Fitting estimator with 12 features.
Fitting estimator with 11 features.
Fitting estimator with 10 features.
Fitting estimator with 9 features.
Fitting estimator with 8 features.
Fitting estimator with 7 features.
Fitting estimator with 6 features.
Fitting estimator with 5 features.
Fitting estimator with 4 features.
Fitting estimator with 3 features.
Fitting estimator with 2 features.
1
Fitting estimator with 32 features.
Fitting estimator with 31 features.
Fitting estimator with 30 features.
Fitting estimator with 29 features

Fitting estimator with 12 features.
Fitting estimator with 11 features.
Fitting estimator with 10 features.
Fitting estimator with 9 features.
Fitting estimator with 8 features.
Fitting estimator with 7 features.
Fitting estimator with 6 features.
Fitting estimator with 5 features.
Fitting estimator with 4 features.
Fitting estimator with 3 features.
Fitting estimator with 2 features.
1
Fitting estimator with 32 features.
Fitting estimator with 31 features.
Fitting estimator with 30 features.
Fitting estimator with 29 features.
Fitting estimator with 28 features.
Fitting estimator with 27 features.
Fitting estimator with 26 features.
Fitting estimator with 25 features.
Fitting estimator with 24 features.
Fitting estimator with 23 features.
Fitting estimator with 22 features.
Fitting estimator with 21 features.
Fitting estimator with 20 features.
Fitting estimator with 19 features.
Fitting estimator with 18 features.
Fitting estimator with 17 features.
Fitting estimator with 16 features

Fitting estimator with 11 features.
Fitting estimator with 10 features.
Fitting estimator with 9 features.
Fitting estimator with 8 features.
Fitting estimator with 7 features.
Fitting estimator with 6 features.
Fitting estimator with 5 features.
Fitting estimator with 4 features.
Fitting estimator with 3 features.
Fitting estimator with 2 features.
1
Fitting estimator with 32 features.
Fitting estimator with 31 features.
Fitting estimator with 30 features.
Fitting estimator with 29 features.
Fitting estimator with 28 features.
Fitting estimator with 27 features.
Fitting estimator with 26 features.
Fitting estimator with 25 features.
Fitting estimator with 24 features.
Fitting estimator with 23 features.
Fitting estimator with 22 features.
Fitting estimator with 21 features.
Fitting estimator with 20 features.
Fitting estimator with 19 features.
Fitting estimator with 18 features.
Fitting estimator with 17 features.
Fitting estimator with 16 features.
Fitting estimator with 15 features

1
Fitting estimator with 32 features.
Fitting estimator with 31 features.
Fitting estimator with 30 features.
Fitting estimator with 29 features.
Fitting estimator with 28 features.
Fitting estimator with 27 features.
Fitting estimator with 26 features.
Fitting estimator with 25 features.
Fitting estimator with 24 features.
Fitting estimator with 23 features.
Fitting estimator with 22 features.
Fitting estimator with 21 features.
Fitting estimator with 20 features.
Fitting estimator with 19 features.
Fitting estimator with 18 features.
Fitting estimator with 17 features.
Fitting estimator with 16 features.
Fitting estimator with 15 features.
Fitting estimator with 14 features.
Fitting estimator with 13 features.
Fitting estimator with 12 features.
Fitting estimator with 11 features.
Fitting estimator with 10 features.
Fitting estimator with 9 features.
Fitting estimator with 8 features.
Fitting estimator with 7 features.
Fitting estimator with 6 features.
Fitting estimator with 5 featu

...

14
Fitting estimator with 32 features.
Fitting estimator with 31 features.
Fitting estimator with 30 features.
Fitting estimator with 29 features.
Fitting estimator with 28 features.
Fitting estimator with 27 features.
Fitting estimator with 26 features.
Fitting estimator with 25 features.
Fitting estimator with 24 features.
Fitting estimator with 23 features.
Fitting estimator with 22 features.
Fitting estimator with 21 features.
Fitting estimator with 20 features.
Fitting estimator with 19 features.
Fitting estimator with 18 features.
Fitting estimator with 17 features.
Fitting estimator with 16 features.
Fitting estimator with 15 features.
14
Fitting estimator with 32 features.
Fitting estimator with 31 features.
Fitting estimator with 30 features.
Fitting estimator with 29 features.
Fitting estimator with 28 features.
Fitting estimator with 27 features.
Fitting estimator with 26 features.
Fitting estimator with 25 features.
Fitting estimator with 24 features.
Fitting estimator with

Fitting estimator with 28 features.
Fitting estimator with 27 features.
Fitting estimator with 26 features.
Fitting estimator with 25 features.
Fitting estimator with 24 features.
Fitting estimator with 23 features.
Fitting estimator with 22 features.
Fitting estimator with 21 features.
Fitting estimator with 20 features.
Fitting estimator with 19 features.
Fitting estimator with 18 features.
Fitting estimator with 17 features.
Fitting estimator with 16 features.
Fitting estimator with 15 features.
14
Fitting estimator with 32 features.
Fitting estimator with 31 features.
Fitting estimator with 30 features.
Fitting estimator with 29 features.
Fitting estimator with 28 features.
Fitting estimator with 27 features.
Fitting estimator with 26 features.
Fitting estimator with 25 features.
Fitting estimator with 24 features.
Fitting estimator with 23 features.
Fitting estimator with 22 features.
Fitting estimator with 21 features.
Fitting estimator with 20 features.
Fitting estimator with 19

14
Fitting estimator with 32 features.
Fitting estimator with 31 features.
Fitting estimator with 30 features.
Fitting estimator with 29 features.
Fitting estimator with 28 features.
Fitting estimator with 27 features.
Fitting estimator with 26 features.
Fitting estimator with 25 features.
Fitting estimator with 24 features.
Fitting estimator with 23 features.
Fitting estimator with 22 features.
Fitting estimator with 21 features.
Fitting estimator with 20 features.
Fitting estimator with 19 features.
Fitting estimator with 18 features.
Fitting estimator with 17 features.
Fitting estimator with 16 features.
Fitting estimator with 15 features.
14
Fitting estimator with 32 features.
Fitting estimator with 31 features.
Fitting estimator with 30 features.
Fitting estimator with 29 features.
Fitting estimator with 28 features.
Fitting estimator with 27 features.
Fitting estimator with 26 features.
Fitting estimator with 25 features.
Fitting estimator with 24 features.
Fitting estimator with

Fitting estimator with 19 features.
Fitting estimator with 18 features.
Fitting estimator with 17 features.
Fitting estimator with 16 features.
Fitting estimator with 15 features.
14
Fitting estimator with 32 features.
Fitting estimator with 31 features.
Fitting estimator with 30 features.
Fitting estimator with 29 features.
Fitting estimator with 28 features.
Fitting estimator with 27 features.
Fitting estimator with 26 features.
Fitting estimator with 25 features.
Fitting estimator with 24 features.
Fitting estimator with 23 features.
Fitting estimator with 22 features.
Fitting estimator with 21 features.
Fitting estimator with 20 features.
Fitting estimator with 19 features.
Fitting estimator with 18 features.
Fitting estimator with 17 features.
Fitting estimator with 16 features.
Fitting estimator with 15 features.
14
Fitting estimator with 32 features.
Fitting estimator with 31 features.
Fitting estimator with 30 features.
Fitting estimator with 29 features.
Fitting estimator with

14
Fitting estimator with 32 features.
Fitting estimator with 31 features.
Fitting estimator with 30 features.
Fitting estimator with 29 features.
Fitting estimator with 28 features.
Fitting estimator with 27 features.
Fitting estimator with 26 features.
Fitting estimator with 25 features.
Fitting estimator with 24 features.
Fitting estimator with 23 features.
Fitting estimator with 22 features.
Fitting estimator with 21 features.
Fitting estimator with 20 features.
Fitting estimator with 19 features.
Fitting estimator with 18 features.
Fitting estimator with 17 features.
Fitting estimator with 16 features.
Fitting estimator with 15 features.
14
Fitting estimator with 32 features.
Fitting estimator with 31 features.
Fitting estimator with 30 features.
Fitting estimator with 29 features.
Fitting estimator with 28 features.
Fitting estimator with 27 features.
Fitting estimator with 26 features.
Fitting estimator with 25 features.
Fitting estimator with 24 features.
Fitting estimator with

In [9]:
# resultado_scores_ranking.sort_values('Mean squared error',ascending = True)

## BMA
|           | p!=0 | EV        | SD      | model 1   | model 2   | model 3   | model 4   | model 5   |
|-----------|------|-----------|---------|-----------|-----------|-----------|-----------|-----------|
| Intercept | 100  | 0.337508  | 0.185   | 0.3375    | 0.3375    | 0.3375    | 0.3375    | 0.3375    |
| x1.x      | 2.1  | 0.004958  | 0.04524 | .         | .         | .         | .         | .         |
| x2.x      | 0.6  | -0.001294 | 0.02336 | .         | .         | .         | .         | .         |
| x3.x      | 0    | 0         | 0       | .         | .         | .         | .         | .         |
| x4.x      | 4.4  | -0.011089 | 0.06623 | .         | .         | .         | .         | .         |
| x5.x      | 0    | 0         | 0       | .         | .         | .         | .         | .         |
| x6.x      | 10.7 | 0.038109  | 0.13069 | .         | .         | .         | .         | .         |
| x7.x      | 0    | 0         | 0       | .         | .         | .         | .         | .         |
| x8.x      | 0    | 0         | 0       | .         | .         | .         | .         | .         |
| x9.x      | 0    | 0         | 0       | .         | .         | .         | .         | .         |
| x10.x     | 0    | 0         | 0       | .         | .         | .         | .         | .         |
| x11.x     | 0    | 0         | 0       | .         | .         | .         | .         | .         |
| x12.x     | 28   | 0.116993  | 0.21659 | .         | .         | 0.4703    | 0.5003    | .         |
| x13.x     | 47.4 | -0.208358 | 0.26194 | .         | .         | .         | .         | -0.4082   |
| x14.x     | 0    | 0         | 0       | .         | .         | .         | .         | .         |
| x15.x     | 0    | 0         | 0       | .         | .         | .         | .         | .         |
| x16.x     | 0    | 0         | 0       | .         | .         | .         | .         | .         |
| x17.x     | 2.6  | -0.007073 | 0.05443 | .         | .         | .         | .         | .         |
| x18.x     | 0    | 0         | 0       | .         | .         | .         | .         | .         |
| x19.x     | 0    | 0         | 0       | .         | .         | .         | .         | .         |
| x20.x     | 8.3  | 0.028627  | 0.11114 | .         | .         | .         | .         | .         |
| x21.x     | 4.5  | 0.014246  | 0.15216 | .         | .         | .         | .         | .         |
| x22.x     | 0    | 0         | 0       | .         | .         | .         | .         | .         |
| x23.x     | 66.4 | 0.34367   | 0.31505 | 0.4799    | 0.5052    | .         | .         | 0.4639    |
| x24.x     | 2.2  | -0.005291 | 0.04533 | .         | .         | .         | .         | .         |
| x25.x     | 82.3 | 0.449576  | 0.28124 | 0.5293    | 0.5303    | 0.6367    | 0.6448    | 0.4588    |
| x26.x     | 0.8  | -0.002981 | 0.03674 | .         | .         | .         | .         | .         |
| x27.x     | 0    | 0         | 0       | .         | .         | .         | .         | .         |
| x28.x     | 0    | 0         | 0       | .         | .         | .         | .         | .         |
| x29.x     | 0    | 0         | 0       | .         | .         | .         | .         | .         |
| x30.x     | 2.4  | 0.009377  | 0.06688 | .         | .         | .         | .         | .         |
| x31.x     | 47.5 | -0.316344 | 0.36156 | .         | -0.6501   | .         | -0.6914   | -0.7172   |
| x32.x     | 49.5 | -0.326244 | 0.35946 | -0.6528   | .         | -0.6893   | .         | .         |
|           |      |           |         |           |           |           |           |           |
| nVar      |      |           |         | 3         | 3         | 3         | 3         | 4         |
| BIC       |      |           |         | -450.0371 | -449.8971 | -449.8544 | -449.7929 | -449.5019 |
| post prob |      |           |         | 0.057     | 0.054     | 0.052     | 0.051     | 0.044     |

## BMA 2:

|           | p!=0 | EV        | SD     | model1    | model2    | model3    | model4    | model5    |
|-----------|------|-----------|--------|-----------|-----------|-----------|-----------|-----------|
| Intercept | 100  | 0.337508  | 0.1725 | 0.3375    | 0.3375    | 0.3375    | 0.3375    | 0.3375    |
| x1.x      | 0    | 0         | 0      | .         | .         | .         | .         | .         |
| x4.x      | 0    | 0         | 0      | .         | .         | .         | .         | .         |
| x6.x      | 2.4  | 0.009218  | 0.0661 | .         | .         | .         | .         | .         |
| x12.x     | 20.8 | 0.082072  | 0.1829 | .         | .         | .         | .         | .         |
| x13.x     | 48.1 | -0.195167 | 0.2396 | .         | -0.4082   | -0.3916   | .         | -0.4412   |
| x17.x     | 0    | 0         | 0      | .         | .         | .         | .         | .         |
| x20.x     | 0    | 0         | 0      | .         | .         | .         | .         | .         |
| x21.x     | 0    | 0         | 0      | .         | .         | .         | .         | .         |
| x23.x     | 80.3 | 0.37002   | 0.2465 | 0.4799    | 0.4639    | 0.438     | 0.5052    | .         |
| x24.x     | 0    | 0         | 0      | .         | .         | .         | .         | .         |
| x25.x     | 96.7 | 0.504406  | 0.2092 | 0.5293    | 0.4588    | 0.4581    | 0.5303    | 0.5107    |
| x30.x     | 0    | 0         | 0      | .         | .         | .         | .         | .         |
| x31.x     | 48   | -0.328505 | 0.3655 | .         | -0.7172   | .         | -0.6501   | .         |
| x32.x     | 52   | -0.353704 | 0.3648 | -0.6528   | .         | -0.7086   | .         | -0.7281   |
|           |      |           |        |           |           |           |           |           |
| nVar      |      |           |        | 3         | 4         | 4         | 3         | 3         |
| BIC       |      |           |        | -434.1567 | -434.2626 | -434.0542 | -433.9955 | -432.6433 |
| postprob  |      |           |        | 0.177     | 0.155     | 0.152     | 0.151     | 0.038     |

## LASSO

In [10]:
from sklearn.linear_model import Lasso
from sklearn.feature_selection import SelectFromModel

In [11]:
for i in np.arange(0.1,1.1,0.1):
    for j in range(1,100):
        X_train, X_test_aux, y_train_aux, y_test_aux = train_test_split(X, y_train, test_size=0.2)
        standarscaler = StandardScaler()
        X_trained_scaled_aux = standarscaler.fit_transform(X_train)
        X_trained_scaled_aux = pd.DataFrame(X_trained_scaled_aux, index=X_train.index, columns=X_train.columns.values)
        
        X_test_scaled_aux = standarscaler.transform(X_test_aux)

        X_test_scaled_aux = pd.DataFrame(X_test_scaled_aux, index=X_test_aux.index, columns=X_test_aux.columns.values)
        Lasso_sel = SelectFromModel(Lasso(alpha=i,max_iter=10000))
        Lasso_sel.fit(X_trained_scaled_aux, y_train_aux)
        name_col = f'Lasso alpha: {i} Iteracion: {j}'
        resultados[name_col] = Lasso_sel.get_support().astype(int)
        variablesLasso = X_trained_scaled_aux.columns[Lasso_sel.get_support()].values
        if len(variablesLasso)>0:
            auxRes= calculated_metrics(variablesLasso,X_trained_scaled,y_train,X_test_scaled,y_test)
            resultado_scores_ranking_cv = pd.concat([resultado_scores_ranking_cv, auxRes])
        print(i)
resultado_scores_ranking_cv.reset_index(inplace = True,drop = True)

0.1
0.1
0.1
0.1
0.1
0.1
0.1
0.1
0.1
0.1
0.1
0.1
0.1
0.1
0.1
0.1
0.1
0.1
0.1
0.1
0.1
0.1
0.1
0.1
0.1
0.1
0.1
0.1
0.1
0.1
0.1
0.1
0.1
0.1
0.1
0.1
0.1
0.1
0.1
0.1
0.1
0.1
0.1
0.1
0.1
0.1
0.1
0.1
0.1
0.1
0.1
0.1
0.1
0.1
0.1
0.1
0.1
0.1
0.1
0.1
0.1
0.1
0.1
0.1
0.1
0.1
0.1
0.1
0.1
0.1
0.1
0.1
0.1
0.1
0.1
0.1
0.1
0.1
0.1
0.1
0.1
0.1
0.1
0.1
0.1
0.1
0.1
0.1
0.1
0.1
0.1
0.1
0.1
0.1
0.1
0.1
0.1
0.1
0.1
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.30000000000000004
0.30000000000000004
0.30000000000000004
0.30000000000000004
0.30000000000000004
0.30000000000000004
0.30000000000000004
0.30000000000000004
0.30000000000000004
0.30000000000000004
0.300000

In [12]:
# resultado_scores_ranking_cv.to_csv('resultado_scores_ranking_cv2.csv',index = False,sep = ';')

In [13]:
resultado_scores_ranking_cv.sort_values('Mean squared error',ascending = True)

,Variables seleccionadas,Mean squared error,R2,Accuray
480,"[x6, x13, x23, x25, x31]",1.569194,0.285763,0.875000
762,"[x1, x2, x6, x13, x20, x23, x25, x31]",2.166063,0.299315,0.791667
654,"[x1, x11, x12, x13, x25, x26, x31]",2.429322,0.264029,0.750000
1135,"[x1, x6, x11, x12, x13, x19, x20, x23, x25, x2...",2.431316,0.061988,0.791667
885,"[x1, x6, x11, x13, x20, x23, x25, x26, x31]",2.457757,0.285325,0.708333
...,...,...,...,...
1241,"[x1, x6, x13, x15, x16, x19, x20, x22, x24, x2...",204.195825,-49.091560,0.541667
1361,"[x1, x6, x8, x10, x13, x15, x20, x21, x22, x24...",209.339926,-25.868538,0.750000
1261,"[x1, x5, x6, x13, x17, x19, x20, x22, x23, x25...",211.584057,-47.842423,0.500000
1278,"[x1, x2, x3, x6, x12, x13, x20, x22, x23, x25,...",225.571410,-61.800336,0.583333


In [14]:
resultados['Modelo Diana 1'] = [0,0,0,1,0,0,1,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,1,1,1,0,0,0,0,0,1,1]
resultados['Modelo Diana 2'] = [0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0]

In [15]:
resultados['Bayes 1'] = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0]
resultados['Bayes 2'] = [0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,1,1]
resultados['Bayes 3'] = [0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,1,0]
resultados['Bayes 3'] = [0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1]
resultados['Bayes 4'] = [1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1]

In [16]:
resultadosFinal = resultados.set_index('VariableName').T.drop_duplicates()

In [17]:
len(resultadosFinal)

1285

In [25]:
resultadosFinal[(resultadosFinal['x1']==0)&(resultadosFinal['x2']==0)
                &(resultadosFinal['x3']==0)&(resultadosFinal['x4']==0)
                &(resultadosFinal['x5']==0)&(resultadosFinal['x6']==0)
                &(resultadosFinal['x7']==0)&(resultadosFinal['x8']==0)
                &(resultadosFinal['x9']==0)&(resultadosFinal['x10']==0)
                &(resultadosFinal['x11']==0)&(resultadosFinal['x12']==1)
                &(resultadosFinal['x13']==0)&(resultadosFinal['x14']==0)
                &(resultadosFinal['x15']==0)&(resultadosFinal['x16']==0)
                &(resultadosFinal['x17']==0)&(resultadosFinal['x18']==0)
                &(resultadosFinal['x19']==0)&(resultadosFinal['x20']==0)
                &(resultadosFinal['x21']==0)&(resultadosFinal['x22']==0)
                &(resultadosFinal['x23']==1)&(resultadosFinal['x24']==0)
                &(resultadosFinal['x25']==1)&(resultadosFinal['x26']==0)
                &(resultadosFinal['x27']==0)&(resultadosFinal['x28']==0)
                &(resultadosFinal['x29']==0)&(resultadosFinal['x30']==0)
                &(resultadosFinal['x31']==0)&(resultadosFinal['x32']==1)]

VariableName,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,...,x23,x24,x25,x26,x27,x28,x29,x30,x31,x32
Lasso alpha: 0.30000000000000004 Iteracion: 29,0,0,0,0,0,0,0,0,0,0,...,1,0,1,0,0,0,0,0,0,1


In [19]:
modelos_torun = resultadosFinal.T

In [20]:
modelos_torun

,Cantidad de variables: 1 Iteracion: 1,Cantidad de variables: 1 Iteracion: 2,Cantidad de variables: 1 Iteracion: 4,Cantidad de variables: 1 Iteracion: 5,Cantidad de variables: 1 Iteracion: 7,Cantidad de variables: 1 Iteracion: 8,Cantidad de variables: 1 Iteracion: 42,Cantidad de variables: 2 Iteracion: 1,Cantidad de variables: 2 Iteracion: 2,Cantidad de variables: 2 Iteracion: 3,...,Lasso alpha: 0.4 Iteracion: 91,Lasso alpha: 0.4 Iteracion: 95,Lasso alpha: 0.4 Iteracion: 98,Lasso alpha: 0.4 Iteracion: 99,Lasso alpha: 0.5 Iteracion: 14,Lasso alpha: 0.5 Iteracion: 15,Lasso alpha: 0.5 Iteracion: 54,Lasso alpha: 0.5 Iteracion: 81,Lasso alpha: 0.5 Iteracion: 91,Modelo Diana 2
VariableName,,,,,,,,,,,,,,,,,,,,,
x1,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
x2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
x3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
x4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
x5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
x6,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
x7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
x8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
x9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [24]:
modelos_torun.to_csv('variablesToProof.csv')

## Sin Cross Validation

In [ ]:
# from sklearn.feature_selection import RFE
# # from sklearn.svm import SVR
# resultados =  pd.DataFrame({'VariableName': X_trained_scaled.columns})
# resultado_scores_ranking = pd.DataFrame()
# for i in range(1,32):
#     name_col = f'Cantidad de variables: {i}'
#     rfe_selector = RFE(estimator=LinearRegression(), n_features_to_select=i, step=1, verbose=5)
#     selector = rfe_selector.fit(X_trained_scaled, y_train['y'].values)
#     resultados[name_col] = selector.support_.astype(int)
    
#     variablesSelected = X_trained_scaled.columns[selector.support_]
#     auxRes= calculated_metrics(variablesSelected,X_trained_scaled,y_train,X_test_scaled,y_test)
#     resultado_scores_ranking = pd.concat([resultado_scores_ranking, auxRes])
#     print(i)
# resultado_scores_ranking.reset_index(inplace = True,drop = True)